# Azure AI Search + Bing Grounding – Workshop Guide
Dieses Notebook zeigt, wie der Workshop-Agent reale Antworten liefert, indem er **Azure AI Search** und **Grounding with Bing Search** kombiniert. Alle Ressourcen werden über `tools_and_data/bing_search/bicep/bing-search.bicep` bereitgestellt.


## Voraussetzungen
- Du hast das Bicep-Skript `bing-grounding-deploy` (oder äquivalent) ausgeführt.
- Du bist per `azd auth login` oder `az login` angemeldet und besitzt Zugriff auf das AI-Projekt.
- In `tools_and_data/.env` sind die Werte aus dem Deployment hinterlegt (Bing Key, Search Endpoint, AI Project Endpoint usw.).
- Inhalte für Azure AI Search pflegst du – falls benötigt – separat über `tools_and_data/vector_db/`.


In [32]:
# Pakete installieren (einmalig pro Notebook-Session)
!uv pip install python-dotenv azure-identity requests

Using Python 3.13.5 environment at: /Users/oscharko/PycharmProjects/Keiko-Evolutio/demo-it-tage-2025/.venv
Audited 3 packages in 1ms


## 1. Umgebung und Variablen laden


In [33]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

ENV_PATH = Path('..') / '..' / '..' / 'tools_and_data' / '.env'
if not ENV_PATH.exists():
    raise FileNotFoundError(f".env nicht gefunden unter {ENV_PATH}")
load_dotenv(ENV_PATH)

# Bing Grounding
bing_resource_name = os.getenv('BING_GROUNDING_RESOURCE_NAME')
bing_endpoint = os.getenv('BING_GROUNDING_ENDPOINT')
bing_api_key = os.getenv('BING_GROUNDING_API_KEY')

# Azure AI Search
search_endpoint = os.getenv('AI_SEARCH_ENDPOINT')
search_index_name = os.getenv('AI_SEARCH_INDEX_NAME', 'docs')

# AI Project (Foundry)
ai_project_account = os.getenv('AI_PROJECT_ACCOUNT_NAME')
ai_project_name = os.getenv('AI_PROJECT_NAME')
project_endpoint = os.getenv('AI_PROJECT_ENDPOINT')
assistant_name = os.getenv('AI_PROJECT_AGENT_NAME', 'bing-grounding-agent')
assistant_model = os.getenv('AI_PROJECT_AGENT_MODEL', 'gpt-4o')
search_connection_name = os.getenv('AI_PROJECT_SEARCH_CONNECTION_NAME', 'conn-search')
bing_connection_name = os.getenv('AI_PROJECT_BING_CONNECTION_NAME', 'conn-bing-grounding')

subscription_id = os.getenv('AZURE_SUBSCRIPTION_ID')
resource_group = os.getenv('AZURE_RESOURCE_GROUP')

print('Konfiguration:')
print(f"  • Bing Grounding: {'OK' if all([bing_resource_name, bing_endpoint, bing_api_key]) else 'FEHLT'}")
print(f"  • Azure AI Search: {'OK' if search_endpoint else 'FEHLT'}")
print(f"  • AI Project Endpoint: {'OK' if project_endpoint else 'FEHLT'}")
print(f"  • Subscription/Resource Group: {'OK' if subscription_id and resource_group else 'FEHLT'}")
if not all([bing_resource_name, bing_endpoint, bing_api_key, search_endpoint, project_endpoint, subscription_id, resource_group, ai_project_account, ai_project_name]):
    raise EnvironmentError('Bitte alle benötigten Variablen in tools_and_data/.env setzen.')


Konfiguration:
  • Bing Grounding: OK
  • Azure AI Search: OK
  • AI Project Endpoint: OK
  • Subscription/Resource Group: OK


## 2. Azure AI Project verbinden
Wir nutzen eine `ChainedTokenCredential`, die zuerst `AzureDeveloperCliCredential` (azd/az Login) und danach `DefaultAzureCredential` ausprobiert.


In [34]:
from azure.identity import AzureDeveloperCliCredential, DefaultAzureCredential, ChainedTokenCredential

credential = ChainedTokenCredential(
    AzureDeveloperCliCredential(),
    DefaultAzureCredential(exclude_shared_token_cache_credential=True,
                           exclude_visual_studio_code_credential=True,
                           exclude_visual_studio_credential=True)
)

def build_connection_id(name: str) -> str:
    return (
        f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/"
        f"Microsoft.CognitiveServices/accounts/{ai_project_account}/projects/{ai_project_name}/connections/{name}"
    )

search_connection_id = build_connection_id(search_connection_name)
bing_connection_id = build_connection_id(bing_connection_name)

print('Connection IDs:')
print('  Azure AI Search:', search_connection_id)
print('  Bing Grounding :', bing_connection_id)


Connection IDs:
  Azure AI Search: /subscriptions/888cfad2-25d5-42f6-b278-d237314b6a19/resourceGroups/rg-workshop-it-tage-2025/providers/Microsoft.CognitiveServices/accounts/ai-services-it-tage-2025/projects/ai-project-it-tage-2025/connections/conn-search
  Bing Grounding : /subscriptions/888cfad2-25d5-42f6-b278-d237314b6a19/resourceGroups/rg-workshop-it-tage-2025/providers/Microsoft.CognitiveServices/accounts/ai-services-it-tage-2025/projects/ai-project-it-tage-2025/connections/conn-bing-grounding


## 3. Hilfsfunktionen für die Agents-API


In [35]:
import time
import requests

API_VERSION = 'v1'
HEADERS_BASE = {"Content-Type": "application/json"}


def get_token() -> str:
    token = credential.get_token('https://ai.azure.com/.default')
    return token.token


def project_request(method: str, path: str, **kwargs):
    url = f"{project_endpoint}{path}"
    headers = dict(HEADERS_BASE)
    headers.update(kwargs.pop('headers', {}))
    headers['Authorization'] = f"Bearer {get_token()}"
    response = requests.request(method, url, headers=headers, timeout=60, **kwargs)
    if not response.ok:
        raise RuntimeError(f"{method} {path} fehlgeschlagen: {response.status_code} {response.text}")
    if response.text:
        return response.json()
    return None


def list_assistants(name: str):
    after = None
    results = []
    while True:
        query = f"/assistants?api-version={API_VERSION}"
        if after:
            query += f"&after={after}"
        data = project_request('GET', query)
        for item in data.get('data', []):
            if item.get('name') == name:
                return item
        if not data.get('has_more'):
            break
        after = data.get('last_id')
    return None


def create_assistant():
    payload = {
        "model": assistant_model,
        "name": assistant_name,
        "instructions": "Nutze Azure AI Search für kuratiertes Wissen (Index docs) und Bing Grounding für aktuelles Webwissen. Antworte mit Quellenangaben.",
        "tools": [
            {"type": "azure_ai_search"},
            {
                "type": "bing_grounding",
                "bing_grounding": {
                    "search_configurations": [
                        {"connection_id": bing_connection_id}
                    ]
                }
            }
        ],
        "tool_resources": {
            "azure_ai_search": {
                "indexes": [
                    {
                        "index_connection_id": search_connection_id,
                        "index_name": search_index_name,
                        "query_type": "semantic"
                    }
                ]
            }
        }
    }
    return project_request('POST', f"/assistants?api-version={API_VERSION}", json=payload)


def ensure_assistant():
    existing = list_assistants(assistant_name)
    if existing:
        return existing
    print('Assistent nicht gefunden – wird erstellt ...')
    created = create_assistant()
    print('✅ Neuer Assistent erstellt.')
    return created


### Assistent prüfen/erstellen


In [36]:
assistant = ensure_assistant()
print('Aktiver Assistent:')
print(json.dumps({
    'id': assistant['id'],
    'name': assistant['name'],
    'model': assistant['model']
}, indent=2))
assistant_id = assistant['id']


Aktiver Assistent:
{
  "id": "asst_LCHgrhTTcrG2PGkNdZyg5w8K",
  "name": "bing-grounding-agent",
  "model": "gpt-4o"
}


## 4. Fragen an den Agenten stellen


In [37]:
import time

def wait_for_run(thread_id: str, run_id: str, poll: float = 2.0):
    while True:
        run = project_request('GET', f"/threads/{thread_id}/runs/{run_id}?api-version={API_VERSION}")
        status = run.get('status')
        if status in {'completed', 'failed', 'requires_action', 'cancelled', 'expired'}:
            if status != 'completed':
                print(f"Run beendet mit Status {status}: {run.get('last_error') or run}")
            return run
        time.sleep(poll)

def extract_message_text(message: dict):
    parts = []
    citations = []
    for block in message.get('content', []):
        block_type = block.get('type')
        if block_type == 'output_text':
            output = block.get('output_text', {})
            for item in output.get('content', []):
                text = item.get('text')
                if isinstance(text, dict):
                    value = text.get('value')
                    if value:
                        parts.append(value)
                elif text:
                    parts.append(text)
            for ann in output.get('annotations', []):
                citation = {
                    'title': ann.get('title') or ann.get('content') or ann.get('reference') or 'Quelle',
                    'url': ann.get('url') or ann.get('link')
                }
                citations.append(citation)
        elif block_type == 'text':
            text_obj = block.get('text')
            if isinstance(text_obj, dict):
                value = text_obj.get('value')
                if value:
                    parts.append(value)
            elif text_obj:
                parts.append(text_obj)
    text = '\n'.join(parts).strip()
    unique_sources = []
    seen = set()
    for cite in citations:
        key = (cite.get('title'), cite.get('url'))
        if key in seen:
            continue
        seen.add(key)
        if cite.get('title') or cite.get('url'):
            unique_sources.append(cite)
    return text, unique_sources

def get_latest_assistant_message(thread_id: str, attempts: int = 10, delay: float = 1.0):
    for _ in range(attempts):
        messages = project_request('GET', f"/threads/{thread_id}/messages?api-version={API_VERSION}&order=desc&limit=10")
        for message in messages.get('data', []):
            if message.get('role') == 'assistant':
                return message
        time.sleep(delay)
    return None

def ask_agent(question: str):
    thread = project_request('POST', f"/threads?api-version={API_VERSION}", json={})
    thread_id = thread['id']
    project_request(
        'POST',
        f"/threads/{thread_id}/messages?api-version={API_VERSION}",
        json={
            'role': 'user',
            'content': [
                {'type': 'text', 'text': question}
            ]
        }
    )
    run = project_request(
        'POST',
        f"/threads/{thread_id}/runs?api-version={API_VERSION}",
        json={'assistant_id': assistant_id}
    )
    run_id = run['id']
    finished = wait_for_run(thread_id, run_id)
    if finished.get('status') != 'completed':
        return None, []
    message = get_latest_assistant_message(thread_id)
    if not message:
        print('⚠️  Keine Assistant-Antwort gefunden (Thread protokolliert).')
        all_messages = project_request('GET', f"/threads/{thread_id}/messages?api-version={API_VERSION}")
        print(json.dumps(all_messages, indent=2))
        return None, []
    answer, sources = extract_message_text(message)
    return answer, sources


### Beispiel: Frage stellen


In [38]:
question = "Was ist Plastizität in der Neurobiologie?"
print(f"Frage: {question}")
answer, sources = ask_agent(question)
print()
print('🧠 Antwort:')
print(answer or 'Keine Antwort erhalten.')
if sources:
    print()
    print('🔗 Quellen:')
    for src in sources:
        title = src.get('title') or 'Quelle'
        url = src.get('url') or ''
        print(f"  - {title} {url}")


Frage: Was ist Plastizität in der Neurobiologie?

🧠 Antwort:
Plastizität in der Neurobiologie, auch als neuronale Plastizität oder Neuroplastizität bekannt, bezieht sich auf die Fähigkeit des Gehirns, seine Struktur und Funktion als Reaktion auf Erfahrungen, Lernprozesse und Umwelteinflüsse zu verändern. Diese Anpassungsfähigkeit ermöglicht es dem Gehirn beispielsweise, neue Fähigkeiten zu erlernen, sich an Verletzungen anzupassen und Gedächtnisinhalte zu speichern.

Neuronale Plastizität wird auf verschiedenen Ebenen beobachtet:

1. **Synaptische Plastizität**: Veränderungen an den Synapsen, den Verbindungsstellen zwischen Neuronen, die die Signalübertragung beeinflussen. Ein bekanntes Beispiel ist die Langzeitpotenzierung (LTP), ein Prozess, bei dem die Effizienz der synaptischen Übertragung langfristig erhöht wird.

2. **Strukturelle Plastizität**: Veränderungen in der Struktur von Neuronen, wie das Wachstum neuer Dendriten oder die Bildung neuer Synapsen. Dies erlaubt eine Neustruk

## 5. Eigene Eingaben
Passe die Liste an, um mehrere Fragen in Folge zu stellen.


In [39]:
questions = [
    "Was ist Bing Grounding?",
    "Wie hilft Azure AI Search im Workshop?"
]

for q in questions:
    print()
    print('=' * 80)
    print('Frage:', q)
    ans, srcs = ask_agent(q)
    print()
    print('Antwort:')
    print(ans or 'Keine Antwort erhalten.')
    if srcs:
        print()
        print('Quellen:')
        for src in srcs:
            title = src.get('title') or 'Quelle'
            url = src.get('url') or ''
            print(f"  - {title} {url}")



Frage: Was ist Bing Grounding?

Antwort:
Bing Grounding ist ein Dienst von Microsoft, der verwendet wird, um Informationen aus dem Web abzurufen und aktuelle Daten zu beschaffen. Dies wird besonders genutzt, wenn spezifische Details oder die neuesten Informationen benötigt werden, die nicht in den bereits verfügbaren Datenquellen enthalten sind. 

Der Dienst ermöglicht es, eine Suchanfrage zu formulieren, relevante Informationen von Webseiten abzurufen und diese Daten in eine kontextuelle Antwort einzubinden. Er wird oft in KI-Systemen genutzt, um sicherzustellen, dass die Antworten aktuell und exakt sind, insbesondere wenn es um zeitkritische Fragen oder um Informationen geht, die sich schnell ändern können.

Zum Beispiel, wenn Sie nach den neuesten Nachrichten zu einem bestimmten Ereignis, aktuellen Wetterinformationen oder aktuellen Markttrends fragen, würde dieser Dienst verwendet, um die entsprechende, aktuelle Information zu liefern.

Frage: Wie hilft Azure AI Search im Workshop

## 6. Weiterführende Nutzung
Der Agent läuft jetzt vollständig auf Azure AI Foundry. Du kannst:
- weitere Threads/Runs erzeugen oder Konversationen fortsetzen,
- über die Foundry-Oberfläche Monitoring & Logging prüfen,
- in `tools_and_data/vector_db/` deinen Azure-AI-Search-Index mit echten Workshop-Dokumenten füttern.


In [40]:
print('=' * 80)
print('NOTEBOOK ABGESCHLOSSEN')
print('=' * 80)
print('Assistent:', assistant_name)
print('Projekt Endpoint:', project_endpoint)


NOTEBOOK ABGESCHLOSSEN
Assistent: bing-grounding-agent
Projekt Endpoint: https://ai-services-it-tage-2025.services.ai.azure.com/api/projects/ai-project-it-tage-2025
